# Hyper Parameter Tuning
- hyper parameter : 모델 설정과 관련해 직접 지정할 수 있는 매개변수
- model parameter : 회귀계수(가중치), 절편 등 모델의 학습 대상이 되는 변수

### GridSearchCV

In [5]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 데이터 로드
iris_input, iris_target = load_iris(return_X_y=True)

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 값
params = {
    'n_neighbors': range(1, 13, 2)
}

# 첫 번째 인자: 모델
# 두 번째 인자: 테스트 할 파라미터 (딕셔너리)
# scoring: 평가 지표 (accuracy, precision, recall, f1)
# cv: 반복 횟수
grid = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid.fit(iris_input, iris_target)

print("최적의 파라미터:", grid.best_params_)
print("최적화된 모델 객체:", grid.best_estimator_)
print("최적화된 점수:", grid.best_score_)

최적의 파라미터: {'n_neighbors': 7}
최적화된 모델 객체: KNeighborsClassifier(n_neighbors=7)
최적화된 점수: 0.9800000000000001


In [6]:
best_knn = grid.best_estimator_
best_knn.fit(iris_input, iris_target)
best_knn.score(iris_input, iris_target)

0.9733333333333334

### RandomSearchCV
- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하는데, 이 범위 중에 랜덤하게 값을 뽑아내 최적의 하이퍼 파라미터 조합을 찾는다.
    - 탐색범위가 넓을 때 짧은 시간 내에 좋은 결과를 얻을 수 있다.
    - 랜덤하게 값을 추출해 계산하므로, 전역 최적값을 놓칠 수 있다.

In [9]:
from sklearn.model_selection import RandomizedSearchCV

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 생성
params = {
    'n_neighbors': range(1, 100, 2)
}

# n_iter: 탐색할 최적의 하이퍼 파라미터 조합 수 (기본값: 10)
#         값이 크면 시간이 오래 걸림 / 값이 작으면 좋은 조합을 찾을 가능성 저하
rd_search = RandomizedSearchCV(knn, params, cv=5, n_iter=10, random_state=0)
rd_search.fit(iris_input, iris_target)

print("최적의 파라미터:", rd_search.best_params_)
print("최적화된 모델 객체:", rd_search.best_estimator_)
print("최적화된 점수:", rd_search.best_score_)
rd_search.cv_results_

최적의 파라미터: {'n_neighbors': 5}
최적화된 모델 객체: KNeighborsClassifier()
최적화된 점수: 0.9733333333333334


{'mean_fit_time': array([0.0006392 , 0.00066476, 0.00020037, 0.00024805, 0.00020266,
        0.00081758, 0.00044451, 0.00049419, 0.00128155, 0.00059991]),
 'std_fit_time': array([0.00052314, 0.00055543, 0.00040073, 0.0004961 , 0.00040531,
        0.00040899, 0.0005459 , 0.00061731, 0.00049877, 0.00048982]),
 'mean_score_time': array([0.00265923, 0.00201139, 0.00130072, 0.00637937, 0.00198269,
        0.00371065, 0.00940289, 0.00595369, 0.00236268, 0.00140066]),
 'std_score_time': array([0.00114053, 0.0008538 , 0.00040235, 0.00119464, 0.00095531,
        0.00146133, 0.00306104, 0.00217624, 0.00174876, 0.00048934]),
 'param_n_neighbors': masked_array(data=[57, 23, 21, 83, 5, 55, 77, 63, 45, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'params': [{'n_neighbors': 57},
  {'n_neighbors': 23},
  {'n_neighbors': 21},
  {'n_neighbors': 83},
  {'n_neighbors': 5},
  {'n_neighbors': 55},
  {'n_neighb